## Split annotated videos into train, validation and test set
This code will split annotated videos into folders for training, validation and testing. To do so it will convert the videos into frames and create a single annotation files containing all annotations for each single frame. Based on the frames the data will then first be shuffled and moved to corresponding folders. The annotation files need to be in coco format. If downloaded from CVAT use COCO Keypoints 1.0 for exporting the keypoints. 

To run this you need to specify the following:
- video_folder: folder that contains videos
- annotation_folder: folder that contains corresponding annotations that should be marked with the same id
- output_frame_path: here the frames will just be saved temporarily before being distributed into the different folders
- train_output_folder
- val_output_folder
- test_output_folder
- train_annotation_combined: file path where combined train annotations will be saved
- val_annotation_combined
- test_annotation_combined
- train, val and test size
- only_test: set this to True if you only want to convert the test data for example into single frames and a combined annotation file



In [16]:
import cv2
import json
import os
import re
import random
import shutil

In [ ]:
only_test = False

In [ ]:
# Specify the folder containing videos and output directories
# Specify the folder where the videos are in
video_folder = "C:\\awilde\\britta\\LTU\\SkiingProject\\SkiersProject\\Data\\annotated_videos\\"
# Specify a folder where frames of videos can be saved in temporarily
output_frame_path = "C:\\awilde\\britta\\LTU\\SkiingProject\\SkiersProject\\Data\\Frames_of_videos\\" 
# Specify the folder where the annotations are in that belong to the  videos
annotation_folder = "C:\\awilde\\britta\LTU\\SkiingProject\\SkiersProject\\Data\\Annotations\\manual_annotations\\" 

# Output folders for train, validation and test data
train_output_folder = "..\\alphapose\\data\\\halpe\\images2\\train2015"
val_output_folder = "..\\alphapose\\data\\\halpe\\images2\\val2017"
test_output_folder = "..\\alphapose\\data\\\halpe\\images2\\test"
train_annotation_combined = "..\\alphapose\\data\\halpe\\annotations2\\combined_train_annotations.json"
val_annotation_combined = "..\\alphapose\\data\\halpe\\annotations2\\combined_val_annotations.json"
test_annotation_combined = "..\\alphapose\\data\\halpe\\annotations2\\combined_test_annotations.json"

if only_test:
    train_size = 0
    val_size = 0
    test_size = 1
else:
    train_size = 0.72
    val_size = 0.18
    test_size = 0.1

seed = 42

# Maximum frame limits for specific videos (optional)
frame_limits = {
    #"DJI_0009.MP4": 890
}

# Initialize counters
frame_counter = 0

In [19]:
# Function to extract the corresponding JSON filename
def get_json_filename(video_name):
    match = re.search(r'DJI_0*(\d+)(_cut)?', video_name)
    if match:
        number = match.group(1).zfill(2)  # Extract the numeric part
        suffix = match.group(2) if match.group(2) else ""  # Extract '_cut' if present
        return f"{number}{suffix}.json"
    return None  # Return None if no match is found

In [20]:
# Ensure output directories exist
if only_test:
    os.makedirs(test_output_folder, exist_ok=True)
else: 
    os.makedirs(train_output_folder, exist_ok=True)
    os.makedirs(val_output_folder, exist_ok=True)
    os.makedirs(test_output_folder, exist_ok=True)

image_id_counter = 1  # Start image IDs at 1
annotation_id_counter = 1  # Start annotation IDs at 1
frame_counter = 0  # Initialize frame counter

video_files = [f for f in os.listdir(video_folder) if f.endswith(('.mp4', '.MP4'))]

all_images = []
all_annotations = []
categories = []

# Process each video and corresponding JSON
for video_file in video_files:
    video_path = os.path.join(video_folder, video_file)
    json_file_name = get_json_filename(video_file)
    json_path = os.path.join(annotation_folder, json_file_name)

    if not os.path.exists(json_path):
        print(f"Warning: JSON file {json_file_name} not found for {video_file}. Path: {json_path} Skipping...")
        continue

    # Split video into frames
    vidcap = cv2.VideoCapture(video_path)
    success, image = vidcap.read()
    current_frame = 0
    max_frames = frame_limits.get(video_file, float('inf'))  # Default to no limit

    while success and current_frame <= max_frames:
        frame_name = f"frame_{frame_counter:06d}.jpg"
        
        # Temporarily save path (actual folder will be assigned after splitting)
        save_path = os.path.join(output_frame_path, frame_name)
        cv2.imwrite(save_path, image)

        # Store image metadata
        all_images.append({
            "id": image_id_counter,
            "width": image.shape[1],
            "height": image.shape[0],
            "file_name": frame_name,
            "license": 0,
            "flickr_url": "",
            "coco_url": "",
            "date_captured": 0
        })

        success, image = vidcap.read()
        frame_counter += 1
        image_id_counter += 1
        current_frame += 1

    # Load JSON annotations
    with open(json_path, 'r') as f:
        data = json.load(f)

    for annotation in data.get("annotations", []):
        annotation["id"] = annotation_id_counter
        annotation["image_id"] = annotation_id_counter  # Align with the correct frame ID
        all_annotations.append(annotation)
        annotation_id_counter += 1

    # Add categories once
    if not categories:
        categories = data.get("categories", [])

# Shuffle images and annotations
combined = list(zip(all_images, all_annotations))
if only_test:
    test_images, test_annotations = zip(*combined)
else:
    random.seed(seed)
    random.shuffle(combined)

    # Split the data
    split_idx_train = int(len(combined) * train_size)
    split_idx_val = int(len(combined) * val_size)
    train_data = combined[:split_idx_train]
    val_data = combined[split_idx_train:split_idx_train+split_idx_val]
    test_data = combined[split_idx_train+split_idx_val:]

    # Separate images and annotations after splitting
    train_images, train_annotations = zip(*train_data)
    val_images, val_annotations = zip(*val_data)
    test_images, test_annotations = zip(*test_data)

# Move images to respective train/val folders
if not only_test:
    for img in train_images:
        src_path = os.path.join(output_frame_path, img["file_name"])
        dest_path = os.path.join(train_output_folder, img["file_name"])
        shutil.move(src_path, dest_path)  # Move image to train folder

    for img in val_images:
        src_path = os.path.join(output_frame_path, img["file_name"])
        dest_path = os.path.join(val_output_folder, img["file_name"])
        shutil.move(src_path, dest_path)  # Move image to validation folder
    
for img in test_images:
    src_path = os.path.join(output_frame_path, img["file_name"])
    dest_path = os.path.join(test_output_folder, img["file_name"])
    shutil.move(src_path, dest_path)  # Move image to validation folder

# Create COCO-style JSONs for train and validation
if not only_test:
    train_json = {
        "licenses": [],
        "info": [],
        "images": list(train_images),
        "annotations": list(train_annotations),
        "categories": categories
    }

    val_json = {
        "licenses": [],
        "info": [],
        "images": list(val_images),
        "annotations": list(val_annotations),
        "categories": categories
    }

test_json = {
    "licenses": [],
    "info": [],
    "images": list(test_images),
    "annotations": list(test_annotations),
    "categories": categories
}

# Save train and validation JSON files
if not only_test:
    with open(train_annotation_combined, 'w+') as f:
        json.dump(train_json, f, indent=4)

    with open(val_annotation_combined, 'w+') as f:
        json.dump(val_json, f, indent=4)
    
with open(test_annotation_combined, 'w+') as f:
    json.dump(test_json, f, indent=4)

print(f"Processed {frame_counter} frames in total.")
if not only_test:
    print(f"Train: {len(train_images)} images, {len(train_annotations)} annotations")
    print(f"Validation: {len(val_images)} images, {len(val_annotations)} annotations")
print(f"Test: {len(test_images)} images, {len(test_annotations)} annotations")

Processed 208 frames in total.
Test: 208 images, 208 annotations


## Verification
This code can be used to check if the annotations and frames have been matched correctly.

In [22]:
# import random
# import cv2

# # Draw annotations on a frame
# def draw_annotations(frame, annotations, categories):
#     for ann in annotations:
#         keypoints = ann.get("keypoints", [])
#         category_id = ann.get("category_id", None)
#         category_name = next((cat["name"] for cat in categories if cat["id"] == category_id), "unknown")
        
#         # Draw keypoints
#         for i in range(0, len(keypoints), 3):
#             x, y, v = keypoints[i:i+3]  # x, y, visibility
#             if v > 0:  # Only draw visible keypoints
#                 cv2.circle(frame, (int(x), int(y)), 5, (0, 255, 0), -1)  # Green keypoints

#         # Optionally, draw lines (skeleton) if available
#         skeleton = categories[0].get("skeleton", [])
#         for connection in skeleton:
#             start_idx, end_idx = connection
#             if start_idx < len(keypoints) // 3 and end_idx < len(keypoints) // 3:
#                 x1, y1, v1 = keypoints[start_idx * 3:start_idx * 3 + 3]
#                 x2, y2, v2 = keypoints[end_idx * 3:end_idx * 3 + 3]
#                 if v1 > 0 and v2 > 0:  # Only draw lines between visible points
#                     cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)  # Blue lines

#     return frame

# # Verify frames by drawing annotations
# def verify_frames_with_annotations(images, annotations, frame_path, categories, num_verifications=5):
#     print("\n--- Verifying Frames with Annotations ---")
#     for _ in range(num_verifications):
#         # Pick a random image
#         image = random.choice(images)
#         frame_file = os.path.join(frame_path, image["file_name"])

#         # Find associated annotations
#         associated_annotations = [ann for ann in annotations if ann["image_id"] == image["id"]]

#         print(f"Verifying Frame: {image['file_name']}, ID: {image['id']}")
#         print("Annotations:")
#         for ann in associated_annotations:
#             print(f"  - ID: {ann['id']}, Keypoints: {ann['keypoints']}")

#         # Load and annotate the frame
#         frame = cv2.imread(frame_file)
#         if frame is not None:
#             annotated_frame = draw_annotations(frame, associated_annotations, categories)

#             # Display the annotated frame
#             cv2.imshow(f"Frame {image['id']} with Annotations", annotated_frame)
#             cv2.waitKey(0)  # Press any key to close the frame window
#             cv2.destroyAllWindows()
#         else:
#             print(f"Warning: Unable to load frame {frame_file}")

# # Call verification after processing all videos
# verify_frames_with_annotations(
#     combined_data["images"], 
#     combined_data["annotations"], 
#     output_frame_path, 
#     combined_data["categories"]
# )